In [44]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [45]:
url = "https://sfbay.craigslist.org/search/sof"

In [46]:
npo_jobs = {}
job_no = 0
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all("p",{"class":"result-info"})
    
    for job in jobs:
        title = job.find('a',{'class':'result-title'}).text
        location_tag = job.find('span',{'class':'result-hood'})
        location = location_tag.text[2:-1] if location_tag else "N/A"
        date = job.find('time',{'class':'result-date'}).text
        link = job.find('a',{'class':'result-title'}).get('href')
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data,'html.parser')
        job_description = job_soup.find('section',{'id':'postingbody'}).text
        job_attributes_tag = job_soup.find('p',{'class':'attrgroup'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else 'N/A'
        
        job_no+=1
        npo_jobs[job_no] = [title,location,date,link,job_attributes,job_description]
        #print('Job Title:', title, '\nLocation:',location,'\nDate:',date,'\nLink:',link, '\nJob Attributes:', job_attributes,'\nJob Description:',job_description,'\n---')
    url_tag = soup.find('a',{'title':'next page'})
    if(url_tag.get('href')):
        url = 'https://sfbay.craigslist.org' + url_tag.get('href')
        print(url)
    else:
        break
print("Total Jobs:",job_no)
npo_df = pd.DataFrame.from_dict(npo_jobs,orient='index',columns = ['Job Title','Location','Date','Link','Job Attributes','Job Description'])

Total Jobs: 105


In [47]:
npo_df.head()

,Job Title,Location,Date,Link,Job Attributes,Job Description
1,Help create a radical Internet Therapy!,berkeley,Jul 17,https://sfbay.craigslist.org/eby/sof/d/berkele...,\ncompensation: $30-$90/hr\n\nemployment type:...,\n\nQR Code Link to This Post\n\n\nWe are hiri...
2,Experienced C/C++ Programmer,santa rosa,Jul 16,https://sfbay.craigslist.org/nby/sof/d/santa-r...,\ncompensation: 60k-85k\n\nemployment type: fu...,\n\nQR Code Link to This Post\n\n\nWe are a so...
3,PCB design engineer for security startup,San Francisco & remote,Jul 15,https://sfbay.craigslist.org/sfc/sof/d/san-fra...,"\ncompensation: Equity only until funding, you...",\n\nQR Code Link to This Post\n\n\nSkills requ...
4,Data Science Fellowship,N/A,Jul 15,https://sfbay.craigslist.org/sfc/sof/d/san-fra...,"\ncompensation: 80,000-160,000\n\nemployment t...",\n\nQR Code Link to This Post\n\n\nWhat is Pat...
5,Product Management Fellowship,N/A,Jul 15,https://sfbay.craigslist.org/sfc/sof/d/san-fra...,"\ncompensation: 80,000-160,000\n\nemployment t...",\n\nQR Code Link to This Post\n\n\nWhat is Pat...


In [49]:
npo_df.to_csv('npo_jobs.csv')